# Udacity Machine Learning Engineer - Capstone Project

Modified code after successful submission.
Refine code and improve UX when entering information.

*Marco Xu, Dec 19th 2020*

## Import Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from IPython.display import display
pd.options.display.max_columns = None
#pd.options.display.float_format = '{:,.2f}'.format

In [3]:
df = pd.read_csv("train.csv")
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


## Data Exploration

In [4]:
df.shape

(3000, 23)

In [7]:
import datetime
#Fixes dates which are in 20xx

def fix_date(x):
    year = x.split('/')[2]
    if int(year) <= 19:
        return x[:-2] + '20' + year
    else:
        return x[:-2] + '19' + year
    
df['release_date'] = df['release_date'].apply(lambda x: fix_date(x))
df['release_date'] = pd.to_datetime(df['release_date'])

In [8]:
min(df.release_date)

Timestamp('1921-01-21 00:00:00')

In [9]:
max(df.release_date)

Timestamp('2017-07-20 00:00:00')

In [15]:
df['release_date_year'] = pd.DatetimeIndex(df['release_date']).year
df['release_date_month'] = pd.DatetimeIndex(df['release_date']).month
df['release_date_quarter'] = pd.DatetimeIndex(df['release_date']).quarter
df['release_date_weekofyear'] = pd.DatetimeIndex(df['release_date']).weekofyear

## Feature Engineering

### Encode variables with several values like genres

In [19]:
import re
from ast import literal_eval
def string_to_list(text,key):
    pattern = "\{(.*?)\}"
    substring = re.findall(pattern, str(text))
    # convert substring to list of dictionaries
    genre_list = [literal_eval('{'+i+'}')[key] for i in substring]
    
    return genre_list

In [20]:
df['genres_cleaned'] = df['genres'].apply(lambda x: string_to_list(x,'name'))
df['spoken_languages_cleaned'] = df['spoken_languages'].apply(lambda x: string_to_list(x,'name'))
df['production_countries_cleaned'] = df['production_countries'].apply(lambda x: string_to_list(x,'name'))
df['production_companies_cleaned'] = df['production_companies'].apply(lambda x: string_to_list(x,'name'))

In [21]:
from sklearn.preprocessing import MultiLabelBinarizer
def list_to_columns(df,column):
    mlb = MultiLabelBinarizer()
    df_encoded = pd.DataFrame(mlb.fit_transform(df[column]),columns=mlb.classes_, index=df.index)
    df = df.join(df_encoded)
    return df

In [22]:
df = list_to_columns(df,"genres_cleaned")
df = list_to_columns(df,"spoken_languages_cleaned")
df = list_to_columns(df,"production_countries_cleaned")
df = list_to_columns(df,"production_companies_cleaned")

### Extract Information from cast

In [23]:
## get top actors
df_top_movies = df.sort_values(by=['revenue'], ascending=False).head(100)
all_actors = df_top_movies['cast'].apply(lambda x: string_to_list(x,'name')).to_list()
top_actors = [actor[:2] for actor in all_actors]

top_actors_distinct = []
for sublist in top_actors:
    for actor in sublist:
        if actor not in top_actors_distinct:
            top_actors_distinct.append(actor)
            
top_actors_distinct
## note: we save this list, since in test data, we don't have revenue information
top_actors_distinct_df = pd.DataFrame(top_actors_distinct,columns=['actor'])
top_actors_distinct_df.to_csv('top_actors_distinct_df.csv', index=False)
top_actors_distinct = pd.read_csv('top_actors_distinct_df.csv').actor.to_list()

In [24]:
# get main_cast
cast_list = df['cast'].apply(lambda x: string_to_list(x,'name')).to_list()
main_cast = [actor[:3] for actor in cast_list]
df['main_cast'] = main_cast

In [25]:
# cast in top_actors included?
def cast_in_top_actors(cast,top_actors_list):
    if not list(set(cast) & set(top_actors_list)): #check if intersection is empty
        return 0
    else:
        return 1
    
    
df['top_actor_included'] = df['main_cast'].apply(lambda x: cast_in_top_actors(x,top_actors_distinct))

In [26]:
## how much did main cast already gross?
df_cast_gross = df[['revenue','main_cast']]
df_cast_gross = list_to_columns(df_cast_gross,"main_cast")
main_cast_list = df_cast_gross.columns.tolist()[2:]

actor_revenue = {}
for actor in main_cast_list:
    df_actor_revenue = df_cast_gross[['revenue',actor]]
    df_actor_revenue = df_actor_revenue.groupby([actor]).sum().reset_index()
    revenue = df_actor_revenue[df_actor_revenue[actor]==1].reset_index().revenue[0]
    actor_revenue[actor] = revenue
    
actor_revenue_df = pd.DataFrame.from_dict(actor_revenue, orient='index',columns=['revenue']).reset_index().rename(columns={'index': 'actor'}) 

In [27]:
## note: we save this df, since in test data, we don't have revenue information
actor_revenue_df.to_csv('actor_revenue_df.csv', index=False)
actor_revenue_df = pd.read_csv('actor_revenue_df.csv')

In [28]:
actor_revenue_group = {}
i = 1
for cast in df.main_cast.tolist():
    sum_revenue = actor_revenue_df[actor_revenue_df.actor.isin(cast)].revenue.sum()
    actor_revenue_group[i] = sum_revenue
    i += 1
    
actor_revenue_group_df = pd.DataFrame.from_dict(actor_revenue_group, orient='index',columns=['revenue']).reset_index().rename(columns={'index': 'id','revenue': 'revenue_cast'})
actor_revenue_group_df = actor_revenue_group_df[['revenue_cast']]

In [29]:
df = df.join(actor_revenue_group_df)

### Extract Information from franchise

In [30]:
## check if film is part of franchise
df['part_of_franchise'] = np.where(df['belongs_to_collection'].isnull(), 0, 1)

In [31]:
# how much did each franchise earn?
franchise_list = df['belongs_to_collection'].apply(lambda x: string_to_list(x,'name')).to_list()
franchise = [np.nan if not franchise else franchise[0] for franchise in franchise_list]
df['franchise'] = franchise

In [32]:
df_franchise_revenue = df.groupby(["franchise"]).sum()[['revenue']].reset_index()
df_franchise_revenue = df_franchise_revenue.rename(columns={'revenue': 'franchise_revenue'})

## note: we save this df, since in test data, we don't have revenue information
df_franchise_revenue.to_csv('df_franchise_revenue.csv', index=False)
df_franchise_revenue = pd.read_csv('df_franchise_revenue.csv')

In [33]:
df = df.merge(df_franchise_revenue, on = 'franchise', how = 'left')
df['franchise_revenue'] = df['franchise_revenue'].fillna(0)

### Production Companies

In [34]:
## is production company part of "big five"? https://en.wikipedia.org/wiki/Major_film_studios
major_film_studios_list = ['Universal Pictures','Paramount Pictures', 'Warner Bros. Pictures', 'New Line Cinema', 
                           'Walt Disney Pictures', '20th Century Studios', 'Columbia Pictures', 'TriStar Pictures']

def is_big_five(productin_company, major_film_studios_list):
    if not list(set(productin_company) & set(major_film_studios_list)): #check if intersection is empty
        return 0
    else:
        return 1
    
    
df['is_big_five'] = df['production_companies_cleaned'].apply(lambda x: is_big_five(x,major_film_studios_list))

### Ratios

In [35]:
df['budget_runtime_ratio'] = (df['budget']/df['runtime']).fillna(0)
df['budget_year_ratio'] = (df['budget']/(df['release_date_year']*df['release_date_year'])).fillna(0)
df['franchise_buget_ratio'] = (df['franchise_revenue']/df['budget']).fillna(0) 
df['cast_buget_ratio'] = (df['revenue_cast']/df['budget']).fillna(0) 

df = df.replace([np.inf, -np.inf], 0)

### Adjust for Inflation

In [36]:
df['inflationBudget'] = df['budget'] + df['budget']*1.8/100*(2018-df['release_date_year'])

In [37]:
df['budget_log'] = np.log1p(df['budget'])

## Modeling

In [38]:
df_small = df.drop(columns=['id', 'belongs_to_collection','genres','homepage','imdb_id','original_title','overview','popularity',
                    'poster_path','production_companies','production_countries','release_date','spoken_languages','status',
                    'tagline','title','Keywords','cast','crew','genres_cleaned','spoken_languages_cleaned','production_countries_cleaned',
                            'main_cast','franchise','production_companies_cleaned','original_language'])


df_small.dropna(inplace=True) #remove rows including na
df_small = df_small.drop(df_small[df_small.budget <= 0].index) #drop rows with zero budget. it's not possible to have movies with zero budget

np.random.seed(23)
df_small = df_small.sample(frac = 1) 

X = df_small.drop(columns=['revenue'])
y = df_small[['revenue']]

In [39]:
X.shape

(2187, 3862)

Training set has many features due to encoding. However, many of them appear few times, so they have almost no variance

### Feature Selection

In [40]:
## remove features with very little variance
from sklearn.feature_selection import VarianceThreshold
threshold = .99 # the higher, the more features
sel = VarianceThreshold(threshold=(threshold * (1 - threshold)))
sel.fit_transform(X)
X = X.iloc[:,sel.get_support()]

In [41]:
X.shape

(2187, 86)

After features with almost no variance have been removed, 86 are left. This is a good number to start modeling

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42)

In [43]:
## save this file: necessary in inference

X_train.head().to_csv('X_head.csv', index=False)

In [44]:
X_train['revenue'] = y_train
X_test['revenue'] = y_test

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [45]:
X_train.to_csv('movies_train.csv', index=False)
X_test.to_csv('movies_test.csv', index=False)

In [46]:
import sagemaker

session = sagemaker.Session() # Store the current SageMaker session
bucket = session.default_bucket()

# S3 prefix (which folder will we use)
prefix = 'box_office_prediction_linear_regression'


# send data to S3. SageMaker will take training data from s3
trainpath = session.upload_data(
    path='movies_train.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

testpath = session.upload_data(
    path='movies_test.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

In [47]:
actor_revenue_df = pd.read_csv('actor_revenue_df.csv')
top_actors_distinct = pd.read_csv('top_actors_distinct_df.csv').actor.to_list()
df_franchise_revenue = pd.read_csv('df_franchise_revenue.csv')
X = pd.read_csv('X_head.csv')

In [48]:
##upload mapping tables to s3
# send data to S3. SageMaker will take training data from s3
actor_revenue_df_path = session.upload_data(
    path='actor_revenue_df.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

top_actors_distinct_path = session.upload_data(
    path='top_actors_distinct_df.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

df_franchise_revenue_path = session.upload_data(
    path='df_franchise_revenue.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

X_path = session.upload_data(
    path='X_head.csv', bucket=bucket,
    key_prefix='sagemaker/sklearncontainer')

First, a simple linear regression serves as base model. The algorithm is simple and serves as benchmark model in many cases.
Second, a random forest regressor is used for training and prediction. Random forests often perform very well and can handle training sets with many inputs. Also it can show feature importance which might be interesting for film producers.
Therefore, it's perfect for our use case.

### Random Forest

In [49]:
%%writefile train_predict_random_forest.py

import numpy as np
from sklearn.ensemble import RandomForestRegressor
import argparse
import pandas as pd
import os
import joblib
from sklearn.metrics import r2_score

def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "rf_model.joblib"))
    return model


def predict_fn(input_data, model):
    print(input_data)
    return model.predict(input_data)

if __name__ =='__main__':

    print('extracting arguments')
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument('--n-estimators', type=int, default=10)
    parser.add_argument('--min-samples-leaf', type=int, default=3)

    # Data, model, and output directories
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file', type=str, default='movies_train.csv')
    parser.add_argument('--test-file', type=str, default='movies_test.csv')
    parser.add_argument('--target', type=str) # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print('reading data')
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print('building training and testing datasets')
    X_train = train_df.drop(columns=[args.target])
    X_test = test_df.drop(columns=[args.target])
    y_train = train_df[args.target]
    y_test = test_df[args.target]

    # train
    print('training model')
    model = RandomForestRegressor(
        n_estimators=args.n_estimators,
        min_samples_leaf=args.min_samples_leaf,
        n_jobs=-1)
    
    model.fit(X_train, y_train)
    
    print(f'R2 on training set: {model.score(X_train, y_train)}')
    
    predictions = model.predict(X_test)
    
    print(f'R2 on test set: {r2_score(y_test,predictions)}')

    # persist model
    path = os.path.join(args.model_dir, "rf_model.joblib")
    joblib.dump(model, path)
    print('model persisted at ' + path)
    
    
    # feature importance
    feature_imp = pd.DataFrame(data= {'feature': X_train.columns, 'importance':model.feature_importances_})
    print(feature_imp.sort_values(by=['importance'], ascending=False))

Overwriting train_predict_random_forest.py


In [50]:
! python train_predict_random_forest.py --n-estimators 50 \
                   --min-samples-leaf 1 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \
                   --target revenue

extracting arguments
reading data
building training and testing datasets
training model
R2 on training set: 0.961069622105539
R2 on test set: 0.7777330947351262
model persisted at ./rf_model.joblib
                      feature  importance
78          franchise_revenue    0.320513
81          budget_year_ratio    0.216113
76               revenue_cast    0.083721
84            inflationBudget    0.058505
0                      budget    0.044134
..                        ...         ...
37                    한국어/조선말    0.000087
59  Metro-Goldwyn-Mayer (MGM)    0.000076
47                     Russia    0.000055
33                     हिन्दी    0.000014
34                      தமிழ்    0.000007

[86 rows x 2 columns]


Random Forest performs better than the benchmark model Linear Regression (higher R-squared on both training and testing set). Also, franchise reveneue, budget and revenue_cast seem to be important features, which is not surprising.

Model performs very well on training but worse on testing set, which is an indicator for overfitting.

### Deploy Random Forest Model with inference pipeline

In [51]:
%%writefile serve/deploy_random_forest.py
import argparse
import joblib
import os
from io import StringIO

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MultiLabelBinarizer
import ast

from utils import select_replace_columns, cast_in_top_actors, list_to_columns, is_big_five

# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

def input_fn(request_body, content_type):
    if content_type == 'text/plain':
    
        movie_dict = ast.literal_eval(request_body.decode('utf-8'))
        print('data imported')
        df = pd.DataFrame.from_dict(movie_dict)
        
        actor_revenue_df = pd.read_csv('s3://sagemaker-eu-central-1-724293319711/sagemaker/sklearncontainer/actor_revenue_df.csv')
        top_actors_distinct = pd.read_csv('s3://sagemaker-eu-central-1-724293319711/sagemaker/sklearncontainer/top_actors_distinct_df.csv')
        df_franchise_revenue = pd.read_csv('s3://sagemaker-eu-central-1-724293319711/sagemaker/sklearncontainer/df_franchise_revenue.csv')
        X = pd.read_csv('s3://sagemaker-eu-central-1-724293319711/sagemaker/sklearncontainer/X_head.csv')

        major_film_studios_list = ['Universal Pictures','Paramount Pictures', 'Warner Bros. Pictures', 'New Line Cinema', 'Walt Disney Pictures',
                                  '20th Century Studios', 'Columbia Pictures', 'TriStar Pictures']


        print('mapping tables loaded')


        df['release_date_year'] = pd.DatetimeIndex(df['release_date']).year
        df['release_date_month'] = pd.DatetimeIndex(df['release_date']).month
        df['release_date_quarter'] = pd.DatetimeIndex(df['release_date']).weekofyear
        df['release_date_weekofyear'] = pd.DatetimeIndex(df['release_date']).weekofyear


        df = list_to_columns(df,"genres")
        df = list_to_columns(df,"spoken_languages")
        df = list_to_columns(df,"production_countries")
        df = list_to_columns(df,"production_companies")


        actor_revenue_group = {}
        i = 1
        for cast in df.main_cast.tolist():
            sum_revenue = actor_revenue_df[actor_revenue_df.actor.isin(cast)].revenue.sum()
            actor_revenue_group[i] = sum_revenue
            i += 1

        actor_revenue_group_df = pd.DataFrame.from_dict(actor_revenue_group, orient='index',columns=['revenue']).reset_index().rename(columns={'index': 'id','revenue': 'revenue_cast'})
        actor_revenue_group_df = actor_revenue_group_df[['revenue_cast']]
        df = df.join(actor_revenue_group_df)
        print('cast revenue')

        df['top_actor_included'] = df['main_cast'].apply(lambda x: cast_in_top_actors(x,top_actors_distinct))

        df['part_of_franchise'] = np.where(df['franchise'].isnull(), 0, 1)

        df = df.merge(df_franchise_revenue, on = 'franchise', how = 'left')
        df['franchise_revenue'] = df['franchise_revenue'].fillna(0)
        print('franchise')

        df['is_big_five'] = df['production_companies'].apply(lambda x: is_big_five(x,major_film_studios_list))

        df['budget_runtime_ratio'] = (df['budget']/df['runtime']).fillna(0)
        df['budget_year_ratio'] = (df['budget']/(df['release_date_year']*df['release_date_year'])).fillna(0)
        df['franchise_buget_ratio'] = (df['franchise_revenue']/df['budget']).fillna(0) 
        df['cast_buget_ratio'] = (df['revenue_cast']/df['budget']).fillna(0) 
        print('ratios')

        df = df.replace([np.inf, -np.inf], 0)
        df['inflationBudget'] = df['budget'] + df['budget']*1.8/100*(2018-df['release_date_year'])
        df['budget_log'] = np.log1p(df['budget'])

        df = select_replace_columns(df,X)


        df_array = np.array(df)
        print(df_array )
        return df_array 
    else:
        print('wrong content_type')



def predict_fn(input_data, model):
    print(input_data)
    return model.predict(input_data)


if __name__ =='__main__':

    print('extracting arguments')
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument('--n-estimators', type=int, default=10)
    parser.add_argument('--min-samples-leaf', type=int, default=3)
    parser.add_argument('--bootstrap', type=bool, default=True)
    parser.add_argument('--max_depth', type=int, default=100)
    parser.add_argument('--min_samples_split', type=int, default=2)
   
    # Data, model, and output directories
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file', type=str, default='movies_train.csv')
    parser.add_argument('--test-file', type=str, default='movies_test.csv')
    parser.add_argument('--target', type=str) # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print('reading data')
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print('building training and testing datasets')
    X_train = train_df.drop(columns=[args.target])
    X_test = test_df.drop(columns=[args.target])
    y_train = train_df[args.target]
    y_test = test_df[args.target]

    # train
    print('training model')
    model = RandomForestRegressor(
        n_estimators=args.n_estimators,
        min_samples_leaf=args.min_samples_leaf,
        max_depth=args.max_depth,
        bootstrap=args.bootstrap,
        min_samples_split=args.min_samples_split,
        n_jobs=-1)
    #print(X_train)
    model.fit(X_train, y_train)

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print('model persisted at ' + path)
    print(args.min_samples_leaf)
    




Overwriting serve/deploy_random_forest.py


In [52]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role

FRAMEWORK_VERSION = '0.23-1'

sklearn_estimator = SKLearn(
    entry_point='deploy_random_forest.py',
    source_dir='serve',
    role = get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.c4.xlarge',
    framework_version=FRAMEWORK_VERSION,
    base_job_name='rf-deployed',
    metric_definitions=[
        {'Name': 'median-AE',
         'Regex': "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters = {'n-estimators': 80,
                       'min-samples-leaf': 3,
                       'max_depth': 200,
                       'bootstrap': True,
                       'min_samples_split': 6,
                       'target': 'revenue'})

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [53]:
sklearn_estimator.fit({'train':trainpath, 'test': testpath})

2020-12-19 16:13:21 Starting - Starting the training job...
2020-12-19 16:13:45 Starting - Launching requested ML instancesProfilerReport-1608394400: InProgress
......
2020-12-19 16:14:46 Starting - Preparing the instances for training...
2020-12-19 16:15:21 Downloading - Downloading input data...
2020-12-19 16:15:47 Training - Downloading the training image..2020-12-19 16:16:01,170 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-12-19 16:16:01,172 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-12-19 16:16:01,182 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-12-19 16:16:01,532 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=70948 sha256=7546da86a33b99221a2103577c3bfd88e30e4c7d2931bcbeb6ad59

In [ ]:
predictor = sklearn_estimator.deploy(instance_type='ml.t2.medium',
                                     initial_instance_count=1)

---------------!

### Test endpoint

In [ ]:
print(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


rf-deployed-2020-12-19-16-16-32-676


In [63]:
import boto3
# create sagemaker client using boto3
client = boto3.client('sagemaker-runtime')

In [64]:
test_data_dict = "{'budget':10000,'main_cast': [['Leonardo DiCaprio','Tom Hardy']],'runtime':120,'release_date':'2019/03/23','genres': [['Action','Crime']],'franchise':'The Avengers Collection','spoken_languages': [['en','jp']],'production_countries': [['United States of America','South Korea']],'production_companies': [['Walt Disney Pictures','Paramount Pictures']]}"

In [65]:
#test_data = "300,10,3"

response = client.invoke_endpoint(
    EndpointName=predictor.endpoint,
    ContentType="text/plain",
    Body=test_data_dict
    )

print(response['Body'].read())

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


b'[221289265.754412]'
